# Question 2.2, SVD

In [60]:
import tensorflow as tf
from matplotlib.pyplot import imshow
import numpy as np
from scipy import linalg
from sklearn.decomposition import RandomizedPCA
import os, os.path
from PIL import Image
from scipy.sparse import lil_matrix
from scipy.stats import truncnorm
%matplotlib inline

In [68]:
size_tup = (256,256)
count = 0
categoryArr = []
sizeArr = []
grayImageArr = []
cumSizeArr = []
indArr = []
animalPath = 'Data/Animal'
fungusPath = 'Data/Fungus'
geoPath = 'Data/Geological Formation'
personPath = 'Data/Person'
plantPath = 'Data/Plant, flora, plant life'
sportPath = 'Data/Sport'

# animal:0, fungus:1, geo:2, person:3, plant:4, sport:5
categoryArr = [animalPath, fungusPath, geoPath, personPath, plantPath, sportPath]
totalSize = 0

for category in categoryArr:
    fold_size = len([name for name in os.listdir(category)])
    sizeArr.append(fold_size)
    totalSize += fold_size
    cumSizeArr.append(totalSize)

# size_arr = [animalSize, fungusSize, geoSize, personSize, plantSize, sportSize]

mat_r = np.empty([totalSize, size_tup[0]*size_tup[1]])
mat_g = np.empty([totalSize, size_tup[0]*size_tup[1]])
mat_b = np.empty([totalSize, size_tup[0]*size_tup[1]])

ind = 0
for a,category in enumerate(categoryArr):
    dirList = os.listdir(category)

    for i,fname in enumerate(dirList):
        img = Image.open(category+'/'+fname)
        img = img.resize(size_tup,Image.ANTIALIAS)
        np_img = np.asarray(img)
        shape_tup = np_img.shape
        r_img = np.empty([size_tup[0],size_tup[1]])
        g_img = np.empty([size_tup[0],size_tup[1]])
        b_img = np.empty([size_tup[0],size_tup[1]])

        # check if image is grayscale
        if len(shape_tup) < 3:
            r_img = np_img
            g_img = np_img
            b_img = np_img
            grayImageArr.append(np_img)
            count += 1
            print fname
        else:
            # iterate through img and store r,g,b
            for color in range(shape_tup[len(shape_tup)-1]):
                if color == 0:
                    r_img = np_img[:,:,color]
                elif color == 1:
                    g_img = np_img[:,:,color]
                else:
                    b_img = np_img[:,:,color]

        r_vect = np.transpose(r_img).flatten()
        g_vect = np.transpose(g_img).flatten()
        b_vect = np.transpose(b_img).flatten()
        mat_r[ind] = r_vect
        mat_g[ind] = g_vect
        mat_b[ind] = b_vect
        indArr.append(ind)
        ind += 1
        

n00015388_45209.JPEG
n00015388_5110.JPEG
n09287968_1537.JPEG
n09287968_1617.JPEG
n09287968_23.JPEG
n09287968_279.JPEG
n09287968_3492.JPEG
n09287968_6184.JPEG
n00007846_15499.JPEG
n00523513_10326.JPEG
n00523513_20172.JPEG
n00523513_22733.JPEG
n00523513_3039.JPEG
n00523513_9073.JPEG


In [9]:
print mat_r.shape, mat_g.shape, mat_b.shape

(8987, 65536) (8987, 65536) (8987, 65536)


In [10]:
# u_r,s_r,v_r = linalg.svd(mat_r)
# u_g,s_g,v_g = linalg.svd(mat_g)
# u_b,s_b,v_b = linalg.svd(mat_b)
pca = RandomizedPCA(n_components=100)
new_mat = pca.fit_transform(mat_r)

In [12]:
print new_mat.shape, ind

(8987, 100) 8987


# Question 3

In [140]:
users = [x for x in range(3000)]
u_c_mat = lil_matrix((len(users), totalSize), dtype=np.float64)
img_ind = 0
for user in users:
    catChoiceArr = list(np.random.choice(6,4,replace=False))
    catChoiceArr.sort()
    choiceInd = 0
    for catInd,catPath in enumerate(categoryArr):
        if choiceInd >= 4:
            break
        if catChoiceArr[choiceInd] == catInd:
            if catInd == 0:
                img_ind = 0
            else:
                img_ind = cumSizeArr[catInd-1]
            m = np.random.geometric(p=0.1,size=1)%sizeArr[catInd] 
            img_indices = list(np.random.choice(sizeArr[catInd],m,replace=False))
            cat_mean = np.random.uniform(1.5,4.5)
            for i,movie_ind in enumerate(img_indices):
                ind_val = img_ind + movie_ind
                rating = truncnorm.rvs(a=(1-cat_mean)/1,b=(5-cat_mean)/1,loc=cat_mean,scale=1)
                u_c_mat[user,ind_val] = rating
                if rating > 5:
                    print rating
            itr_count += 1
            choiceInd += 1
    

In [144]:
p,q,r = linalg.svd(np.transpose(u_c_mat.todense()))

In [143]:
print p.shape, q.shape, r.shape

(3000, 3000) (3000,) (8987, 8987)
